## Bibliotecas

In [0]:
import pandas as pd
from great_expectations.dataset import SparkDFDataset
from datetime import datetime, timedelta

## Leitura da extração

In [0]:
data = str(datetime.now().date())
df_posicao_estoque = spark.read.csv(f"/mnt/adfraw/posicao_estoque_{data}_raw.csv", sep=";", header=True)
df_posicao_estoque.display()

DATA,CMV,ESTOQUE_VALOR,ESTOQUE_QTDE,VENDA_VALOR,VENDA_QTDE,MARGEM_FINAL,VENDA_LIQUIDA_FINAL,PRODUTO,LOJA


## Parâmetros do Great Expectations

### Spark DF

In [0]:
raw_test_df = SparkDFDataset(df_posicao_estoque)
type(raw_test_df)

Out[59]: great_expectations.dataset.sparkdf_dataset.SparkDFDataset

### Colunas importantes

In [0]:
colunas_importantes = [
    "DATA",
    "CMV",
    "ESTOQUE_VALOR",
    "ESTOQUE_QTDE",
    "VENDA_VALOR",
    "VENDA_QTDE",
    "MARGEM_FINAL",
    "VENDA_LIQUIDA_FINAL",
    "PRODUTO",
    "LOJA"
]

## Análise de existência de colunas

In [0]:
def coluna_existe(colunas_importantes):
    
    for column in colunas_importantes:            
        resultado = raw_test_df.expect_column_to_exist(column)            
        if(resultado.success == True):                
            pass;                
        else:                
            return False;
            break;                
    return True;

valida_coluna_existe = coluna_existe(colunas_importantes)

print(valida_coluna_existe)

True


## Análise de dados nulos

In [0]:
def coluna_nula(colunas_importantes):
    
    for column in colunas_importantes:            
        resultado = raw_test_df.expect_column_values_to_not_be_null(column)            
        if(resultado.success == True):                
            pass;                
        else:
            return False;
            break;                
    return True;

valida_coluna_nula = coluna_nula(colunas_importantes)

print(valida_coluna_nula)

True


## Análise de Data

In [0]:
data_ontem = str(datetime.now().date() - timedelta(days=2))
print(data_ontem)

2023-03-29


In [0]:
def valida_data(data_ontem):
    
    resultado = raw_test_df.expect_column_values_to_be_in_set("DATA", [data_ontem])
    if(resultado.success == True):
        return True;
    else:
        return False;

valida_data = valida_data(data_ontem)
print(valida_data)

True


## Valida contagem de linhas

In [0]:
sql_limites = """SELECT
  NTILE,
  MIN(CONTAGEM) AS LIMITE_INFERIOR,
  MAX(CONTAGEM) AS LIMITE_SUPERIOR,
  COUNT(DATA) AS OCORRENCIAS
FROM
  (SELECT
    DATA,
    COUNT(*) AS CONTAGEM,
    NTILE(1) OVER (ORDER BY COUNT(*)) AS NTILE
  FROM
    raw.posicao_estoque_main_raw
  WHERE
    CAST(DATA AS DATE) >= DATEADD(MONTH, -6, CAST(DATA AS DATE))
  GROUP BY DATA) a
GROUP BY 1"""

df_limites = spark.sql(sql_limites).toPandas()

In [0]:
limite_inferior = df_limites['LIMITE_INFERIOR'].values[0]
limite_superior = df_limites['LIMITE_SUPERIOR'].values[0]
print(limite_inferior, limite_superior)

87393 118857


In [0]:
def valida_contagem(limite_inferior, limite_superior):
    
    resultado = raw_test_df.expect_table_row_count_to_be_between(limite_inferior, limite_superior)
    
    if(resultado.success == True):
        return True;
    else:
        return False;

valida_contagem = valida_contagem(limite_inferior, limite_superior)

print(valida_contagem)

False


## Validação Final

In [0]:
validacoes = [valida_coluna_existe,
    valida_coluna_nula,
    valida_data,
    valida_contagem]

def validacao_final(validacoes):
    if False in validacoes:
        return False;
    else:
        return True;
    
validacao_final = validacao_final(validacoes)
print(validacao_final)

False


## Salvando Dados

In [0]:
tabela = "posicao_estoque_raw"
camada = "raw"
data_agora_timestamp = str(datetime.now())
data_agora = str(datetime.now().date())

In [0]:
if validacao_final == False:
    posicao_estoque_erro = df_posicao_estoque.toPandas()
    posicao_estoque_erro.to_csv(f'/dbfs/mnt/erros_dataquality/posicao_estoque_{data}.csv', sep=";", index=False)
    pasta_arquivo = f'/dbfs/mnt/erros_dataquality/posicao_estoque_{data}.csv'
else:
    pasta_arquivo=''

In [0]:
dicionario_validacao = [{"tabela":tabela,
                       "camada":camada,
                       "todas_as_colunas":valida_coluna_existe,
                       "dados_vazios":valida_coluna_nula,
                       "data_valida":valida_data,
                       "contagem":valida_contagem,
                       "aprovado":validacao_final,
                       "mnt":pasta_arquivo,
                       "data_validacao":data_agora,
                       "timestamp":data_agora_timestamp}]

df_validacao = spark.createDataFrame(pd.DataFrame(dicionario_validacao))
df_validacao.display()

tabela,camada,todas_as_colunas,dados_vazios,data_valida,contagem,aprovado,mnt,data_validacao,timestamp
posicao_estoque_raw,raw,true,true,true,false,false,/dbfs/mnt/erros_dataquality/posicao_estoque_2023-03-31.csv,2023-03-31,2023-03-31 01:01:31.804553


In [0]:
df_validacao.write \
    .option('path', '/mnt/dataquality/valida_extracao') \
    .partitionBy(['data_validacao']) \
    .mode('append') \
    .saveAsTable('dataquality.valida_extracao')

In [0]:
if validacao_final == False:
    raise Excepetion("Data Quality Posição de Estoque RAW não foi aprovada")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-333799256638454> in <cell line: 1>()
      1 if validacao_final == False:
----> 2     raise Excepetion("Data Quality Posição de Estoque RAW não foi aprovada")

NameError: name 'Excepetion' is not defined